# Validate Networks with passing in entire test set or individual selected images

The performance of the network might look better than it really is when evaluated on batches of the entire dataset. To solve this, an harder validation set is used which mainly contains samples from the harder to classify data set (simulations with thicknesses close to the threshold)

--> Make sure to simulate an test data set which has not been used before for andy kind of training. The test set I used in my folder structure is more a validation set because I used it to tune the hyper parameters.

Import necessary modules first

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

use cell below for import of matplotlib only if you want to plot for latex with .pgf later

In [3]:
import matplotlib
# comment this in if you want to export to latex
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [4]:
data_path = Path.cwd() / 'data'
checkpoint_path = Path.cwd() / 'model_checkpoints'
trained_path = Path.cwd() / 'trained_models'
# sims_path = Path.cwd().parent.resolve() / 'analysis_2dfft'
sims_path = Path('C:\\Users\\Max\\Documents') / 'analysis_2dfft'

In [5]:
from dl_code.testing import Tester
from dl_code.my_resnet import MyResNet18, MyResNet34
from dl_code.simple_net_big import SimpleNetBig

from dl_code.data_transforms import get_all_transforms
from dl_code.confusion_matrix import generate_and_plot_confusion_matrix

## Load network and testing class

In [6]:
# inp_size = (512, 512)
inp_size = (1024, 1024)
dataset_mean, dataset_std = 0.3150067262879628, 0.1554323642999201

is_simple_net = True

if is_simple_net:
    simple_net_big = SimpleNetBig() # comment in leaky layer is runtime error
    tester = Tester(
        data_dir = data_path,
        data_set = 'test',  #'train',  # "test",
        model = simple_net_big,  #my_resnet, #simple_net_big,
        is_checkpoint = False,
        load_trained_model = True,
        save_dir = trained_path,  # checkpoint_path / 'resnet18_cluster',
        model_name = '02-04_01-13-40_trained_SimpleNetBig_final.pt', #'01-15_01-13-45_trained_SimpleNetBig_final.pt', # '01-13_16-43-56_trained_SimpleNetBig_final.pt', ##'01-11_10-16-03_trained_SimpleNetBig_final.pt',  # '01-10_20-47-01_trained_SimpleNetBig_final.pt',  #'12-07_15-47-20_trained_SimpleNetBig_final.pt',  #'01-07_19-13-04_trained_MyResNet18_final.pt',#'12-07_15-47-20_trained_SimpleNetBig_final.pt', #'01-07_19-13-04_trained_MyResNet18_final.pt', #'12-06_00-46-09_trained_SimpleNetBig_final.pt',  # "11-18_13-57-46_trained_MyResNet34_final.pt",  #'trained_MyResNet18_final.pt',  #,'trained_SimpleNetBig_final_v1.pt',  #'checkpoint.pt',
        data_transforms = get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
        sims_path = sims_path,
        batch_size = 1, #32,
        cuda = True,
    )

In [10]:
if not is_simple_net:
    my_resnet = MyResNet18()
    # my_resnet = MyResNet34()
    tester = Tester(
        data_dir = data_path,
        data_set = 'train',  # "test",
        model = my_resnet,  #my_resnet, #simple_net_big,
        is_checkpoint = True,
        load_trained_model = True,
        save_dir = checkpoint_path / 'resnet18_cluster',  # trained_path,  # checkpoint_path / 'resnet18_cluster',
        model_name = 'checkpoint.pt',  # '01-10_20-47-01_trained_SimpleNetBig_final.pt',  #'12-07_15-47-20_trained_SimpleNetBig_final.pt',  #'01-07_19-13-04_trained_MyResNet18_final.pt',#'12-07_15-47-20_trained_SimpleNetBig_final.pt', #'01-07_19-13-04_trained_MyResNet18_final.pt', #'12-06_00-46-09_trained_SimpleNetBig_final.pt',  # "11-18_13-57-46_trained_MyResNet34_final.pt",  #'trained_MyResNet18_final.pt',  #,'trained_SimpleNetBig_final_v1.pt',  #'checkpoint.pt',
        data_transforms = get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
        sims_path = sims_path,
        batch_size = 1, #32,
        cuda = True,
    )

## Run a batch of images trough the Network

Function only makes sense when batch size is 1

In [6]:
acc, loss, probs, y = tester.push_batch()

print(f'accuracy of batch --> = {acc}\n'
      f'loss of batch ------> = {loss}\n'
      f'true labels --------> = {y}\n'
      f'probabilities ------> = \n{probs}\n'
      )

accuracy of batch --> = 0.0
loss of batch ------> = 0.7550636529922485
true labels --------> = [1]
probabilities ------> = 
[[0.53001934 0.46998066]]



## Run just a single example through the Network

Select an index smaller or equal the length of the dataset as below:

In [11]:
dataset_length = tester.dataset_length
print(f'dataset length = {dataset_length}')

dataset length = 122


In [6]:
idx = 90

gt_c, p_c, prob, thick, gd = tester.push_single(idx)

print(
    f'Thickness ----------------->: {thick} microns\n'
    f'Gap_depth ----------------->: {gd} microns\n'
    f'Predicted class of sample ->: {int(p_c)}\n'
    f'True class of sample ------>: {gt_c}\n'
    f'probability of prediction ->: {"{:.3f}".format(float(prob.ravel()))}\n'
)

print(type(gt_c), type(p_c), type(prob), type(thick), type(gd))

Thickness ----------------->: 290.0 microns
Gap_depth ----------------->: 75.0 microns
Predicted class of sample ->: 0
True class of sample ------>: 0
probability of prediction ->: 0.555

<class 'int'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'float'> <class 'float'>


# Push whole test set though network and get labels for plotting later:
This is a necessary step for plotting later

In [7]:
f_all = tester.push_all()

C:\Users\Max\OneDrive - Georgia Institute of Technology\Documents\Uni Gatech MSC\A Lab Research Wave CEE\python_scripts\dl_code\testing.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f[idx, :] = np.array(self.push_single(idx))


## Extract ground truth Thickness

Extract the real thickness for all elements in
__test set__!

In [7]:
# from create_model.visualize_dataset import get_thickness_info

thick_threshold = 204

features = np.zeros((tester.dataset_length, 2))
real_thicks = []
for idx in range(tester.dataset_length):
    thick, gd = tester.get_simulation_information(idx, sims_path)
    features[idx, 0] = thick * 1E-6
    features[idx, 1] = gd * 1E-6
    real_thicks.append(thick - gd)

# get_thickness_info(features, thick_threshold, save=False)

## Plot the prediction data

Create a scatter plot with gap depth over thickness, color the data points according to their prediction and adjust the size of the dots according to the accuracy

In [8]:
thick_threshold = 200

f = f_all[:-1,:]
# print(f)

fig = plt.figure(1, dpi=200)
fig.set_size_inches(w=6, h=4)
ax = plt.gca()

# print(f)

f_color = ['g' if elem[1] == 1 else 'r' for elem in f]
f_markers = ["D" if elem[1] == 1 else "o" for elem in f]
f_size = [50 if elem[1] == 1 else 50 for elem in f]
# ax.scatter(f[:, 3], f[:, 4], s=40, c=f_color, alpha=0.8) # s=f[:, 2]*100
for idx in range(len(f_markers)):
    ax.scatter(f[idx, 3], f[idx, 4], s=f_size[idx],
               marker=f_markers[idx], c=f_color[idx], alpha=0.8) # s=f[:, 2]*100

x = np.linspace(0, 600, 100)
# y = x - thick_threshold
y = 2*np.ones((len(x),1))
ax.plot(x, y, c='k')

# import pdb; pdb.set_trace()

plt.axis([-10, 610, -10, 210])
# plt.title(f'CNN classification for test set\n'
#           f'12/06/2021 v2')
plt.xlabel(r'Coating thickness in $\mu$m (1E-6m)')
plt.ylabel(r'Gap depth in $\mu$m (1E-6m)')

date = '02042022_' # '12062021'
net = 'SimpleNetBig' #'ResNet18' # 'SimpleNetBig'
if True:
    plt.savefig(f'./figures/{date}_test_set_classification_{net}_v4.png', dpi=200)
    plt.savefig(f'./figures/{date}_test_set_classification_{net}_v4.pgf',
                 bbox_inches='tight', backend='pgf', format='pgf', dpi=100)
plt.show()
plt.close(1)

<ipython-input-8-859a8a178c48>:39: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Create Confusion Matrix

In [16]:
generate_and_plot_confusion_matrix(simple_net_big, tester.dataset,
                                   save_matrix=True, use_cuda=True)


RuntimeError: CUDA out of memory. Tried to allocate 1.96 GiB (GPU 0; 4.00 GiB total capacity; 2.23 GiB already allocated; 69.90 MiB free; 2.34 GiB reserved in total by PyTorch)